# Teste técnico - Eng. de Dados (MINEHR)

 > Candidato: Flávio Macaúbas Torres Filho
 

### Dados originais

* Disponíve aqui: [Dados](https://github.com/FlavioMacaubas/desafio_minerh/blob/master/base_mensalizada_de_funcionarios.csv)

### Dados após ajustes

* Podem ser acessados aqui: [Dados](https://community.cloud.databricks.com/?o=1934713391467732#table/default/base_mensalizada_de_funcionarios_csv)
  + Necessita login no DataBricks

* Alternativamente: [Github](https://github.com/FlavioMacaubas/desafio_minerh/blob/master/dados_finais.csv)


### Conclusões Preliminares

* Foram perdidas 16 observações (13,2%) nas etapas solicitadas;

* A etapa de categorização do Grau de Instrução foi antecipada dado as especificações do item 4 do 1º Passo.

* Foi percebido uma incosistência entre Mês Referência e Data de Admissão na etapa de transformação.

  + Recomendação: a checagem da data de admissão do funcionário 24457
  
### Insights Preliminares

* A idade média dos colaboradores é de 35 anos

* O tempo de empresa médio é de 117 meses - aproximadamente 10 anos

* A média salarial é de aproximadamente R$ 5000,00

* Houve 9 mudanças salariais

* Houve 2 mudanças de cargos

In [0]:
# Imports necessários

# Funções para "tipagem" do esquema
from pyspark.sql.types import StringType, DoubleType, IntegerType

# Funções utilizadas
from pyspark.sql.functions import (
                                   col, # Facilitação de acesso as colunas
                                   when, # Condicional
                                   count, # Para contagem
                                   to_date, # Tratamento especial para datas formato dd-mm-yyyy
                                   trim, # Retirar espaços em brancos
                                   udf, # Para criação de funções do usuário 
                                   month, # Extração de meses de datas
                                   year, # Extração de anos de datas
                                   months_between, # Para calculo da idade
                                   floor, # Arrendodamento para baixo
                                   round, # Arredondamento mais próximo
                                   lit, # Valores literais
                                   lag, # Defasagens
                                   sum, # Somatório
                                  ) 

# Definição de ordem
from pyspark.sql.window import Window

# Leitura dos dados

In [0]:
# Localização do arquivo de dados e tipo
file_location = "/FileStore/tables/base_mensalizada_de_funcionarios.csv"
file_type = "csv"

# Opcões do CSV 
infer_schema = 'false' # Deixarei como falso propositalmente
first_row_is_header = "true"
delimiter = ";"

# Fazendo leitura dos dados com especificações definidas
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

# Cópia do dataframe para checagem final
df_original = df.alias('df_original')

display(df)

mes_referencia,id_funcionario,data_de_nascimento,data_de_admissao,data_de_demissao,grau_de_instrucao,cargo,salario
01/01/2020,11223,01/04/1995,01/06/2018,null,4ª série completo,operador de colheita I,1500
01/01/2020,13456,05/03/1960,11/02/2002,null,ensino médio comp,assistente de rh,1400
01/01/2020,12345,02/11/1980,null,null,ensino fundamental completo,operador de colheita II,1800
01/01/2020,24457,26/07/2000,10/05/2020,null,mestrado,gerente de produção,11500
01/01/2020,13254,22/05/1977,03/09/2005,null,ensino medio incompleto,operador de colheita I,1625
01/01/2020,57687,28/05/1991,18/10/2015,null,doutorado,cientista de dados,7200
01/01/2020,46578,11/01/1972,26/11/2009,null,ensino superior completo,analista de sistemas pleno,7500
01/01/2020,46578,11/01/1972,26/11/2009,null,ensino superior completo,analista de sistemas pleno,7500
01/01/2020,59393,17/05/1998,08/01/2019,null,ensino médio completo,assistente administrativo,1400
01/01/2020,95847,23/04/1993,25/04/2012,null,graduação,líder de tecnologia,11800


In [0]:
# Verificando esquema da base
df.printSchema()

root
 |-- mes_referencia: string (nullable = true)
 |-- id_funcionario: string (nullable = true)
 |-- data_de_nascimento: string (nullable = true)
 |-- data_de_admissao: string (nullable = true)
 |-- data_de_demissao: string (nullable = true)
 |-- grau_de_instrucao: string (nullable = true)
 |-- cargo: string (nullable = true)
 |-- salario: string (nullable = true)



In [0]:
# Contando o número de observações nos dados
print(f"Nº observações: {df.count()}")

Nº observações: 121


# Tratando os dados

-----
* **Completude**: em nossas bases, em cada mês, colunas como data de admissão, data de nascimento, id do funcionário, grau de instrução, cargo e salário não podem conter valores nulos.

In [0]:
# Contando quantidade de valores ausentes por coluna
df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+--------------+--------------+------------------+----------------+----------------+-----------------+-----+-------+
|mes_referencia|id_funcionario|data_de_nascimento|data_de_admissao|data_de_demissao|grau_de_instrucao|cargo|salario|
+--------------+--------------+------------------+----------------+----------------+-----------------+-----+-------+
|             0|             0|                 3|               4|             119|                0|    0|      0|
+--------------+--------------+------------------+----------------+----------------+-----------------+-----+-------+



É possível verificar que das colunas de interesse, apenas data de nascimento e data de admissão possuem valores ausentes. Queremos retirar esses valores, para isso:

In [0]:
# Colunas de interesse de remoção de valores ausentes
colunas_interesse = ['data_de_nascimento', 'data_de_admissao']

# Retirando valores ausentes das colunas de interesse
df = df.na.drop(subset = colunas_interesse)

# Reverificando contagem
df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+--------------+--------------+------------------+----------------+----------------+-----------------+-----+-------+
|mes_referencia|id_funcionario|data_de_nascimento|data_de_admissao|data_de_demissao|grau_de_instrucao|cargo|salario|
+--------------+--------------+------------------+----------------+----------------+-----------------+-----+-------+
|             0|             0|                 0|               0|             114|                0|    0|      0|
+--------------+--------------+------------------+----------------+----------------+-----------------+-----+-------+



In [0]:
# Contando número de observações nos dados após remoção de valores ausentes
print(f"Nº observações: {df.count()}")

Nº observações: 115


Percebe-se uma redução de 6 observações - de 121 para 115 - nesta primeira etapa de limpeza dos dados.

------

* **Unicidade**: colunas que são chaves não podem se repetir dentro de um mês referência, o que quer dizer que os dados de um funcionário não podem aparecer mais de uma vez dentro de um mês.

Objetivamente, nós queremos evitar dupla contagem de funcionários em um mesmo mês, em decorrência iremos procurar pro valores duplicados na coluna id_funcionario por mês.

In [0]:
# Verificando valores duplicados
df.exceptAll(df.dropDuplicates(subset = ['mes_referencia', 'id_funcionario'])).show()

+--------------+--------------+------------------+----------------+----------------+--------------------+--------------------+-------+
|mes_referencia|id_funcionario|data_de_nascimento|data_de_admissao|data_de_demissao|   grau_de_instrucao|               cargo|salario|
+--------------+--------------+------------------+----------------+----------------+--------------------+--------------------+-------+
|    01/06/2020|         46457|        12/02/1982|      07/11/1998|            null|       pós graduação|      analista de BI|   5900|
|    01/01/2020|         46578|        11/01/1972|      26/11/2009|            null|ensino superior c...|analista de siste...|   7500|
|    01/05/2020|         13254|        22/05/1977|      03/09/2005|            null|ensino medio inco...|operador de colhe...|   1625|
|    01/07/2020|         57687|        28/05/1991|      18/10/2015|            null|           doutorado|  cientista de dados|   7200|
|    01/04/2020|         13456|        05/03/1960|     

In [0]:
# Removendo duplicados que ocorram no mesmo mês
df = df.dropDuplicates(subset = ['mes_referencia', 'id_funcionario'])

# Quantidade de observçaões após remoção dos duplicados
print(f"Nº observações: {df.count()}")

Nº observações: 110


Houve a perda de 5 observações ao remover os funcionários repetidos no mesmo mês.


------

* **Integridade**: temos que garantir que em determinada coluna haja valores condizentes ao tipo e à finalidade dela. Por exemplo: se a coluna é do tipo texto, não pode haver valores incompatíveis como, por exemplo, números. Garanta a integridade dos dados em cada coluna.

Nesta etapa, quero garantir a tipagem correta dos dados, para isto irei converter as colunas ao DataTypes desejados, eliminando aqueles elementos que não satisfaçam as condições.

**Importante**: se algum valor não for "castável", a função cast atribuirá None. Este comportamento é particularmente útil pois assim podemos retirar dos nossos dados aquelas observações incosistentes. Em outras palavras, é um atalho para garantir integridade da coluna, porém é de crucial importância revisitar os dados para garantir que tudo ocorreu de acordo com o planejado.

In [0]:
# Convertendo as colunas desejas
df = df.withColumn('mes_referencia', to_date(col('mes_referencia'), "dd/MM/yyyy").alias('mes_referencia')) \
        .withColumn('data_de_nascimento', to_date(col('data_de_nascimento'), "dd/MM/yyyy").alias('data_de_nascimento')) \
        .withColumn('data_de_admissao', to_date(col('data_de_admissao'), "dd/MM/yyyy").alias('data_de_admissao')) \
        .withColumn('data_de_demissao', to_date(col('data_de_demissao'), "dd/MM/yyyy").alias('data_de_demissao')) \
        .withColumn('salario', col('salario').cast(DoubleType()))
                
# Verificando esquema
df.printSchema()

root
 |-- mes_referencia: date (nullable = true)
 |-- id_funcionario: string (nullable = true)
 |-- data_de_nascimento: date (nullable = true)
 |-- data_de_admissao: date (nullable = true)
 |-- data_de_demissao: date (nullable = true)
 |-- grau_de_instrucao: string (nullable = true)
 |-- cargo: string (nullable = true)
 |-- salario: double (nullable = true)



In [0]:
# Verificando presença de valores ausentes após o cast
df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+--------------+--------------+------------------+----------------+----------------+-----------------+-----+-------+
|mes_referencia|id_funcionario|data_de_nascimento|data_de_admissao|data_de_demissao|grau_de_instrucao|cargo|salario|
+--------------+--------------+------------------+----------------+----------------+-----------------+-----+-------+
|             0|             0|                 0|               0|             109|                0|    0|      1|
+--------------+--------------+------------------+----------------+----------------+-----------------+-----+-------+



Lembre-se que a coluna de data_de_demissao não é alvo da nossa investigação de valores ausentes, pois se um funcionário ainda faz parte da empresa, naturalmente não há data de demissão. Porém,
agora possuímos um observação ausente na coluna de salários e queremos removê-lá.

In [0]:
# Retirando valores ausentes das colunas de interesse
df = df.na.drop(subset = ['salario'])

# Reverificando contagem
df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+--------------+--------------+------------------+----------------+----------------+-----------------+-----+-------+
|mes_referencia|id_funcionario|data_de_nascimento|data_de_admissao|data_de_demissao|grau_de_instrucao|cargo|salario|
+--------------+--------------+------------------+----------------+----------------+-----------------+-----+-------+
|             0|             0|                 0|               0|             108|                0|    0|      0|
+--------------+--------------+------------------+----------------+----------------+-----------------+-----+-------+



In [0]:
# Quantidade de observçaões após remoção dos duplicados
print(f"Nº observações: {df.count()}")

Nº observações: 109


Temos uma observação a menos nos nossos dados.

--------
* **Padronização de categorias**: colunas de categorias, não podem conter outros valores além das categorias previstas para ela.

Naturalmente, temos que as colunas categóricas dos dados serão "grau_de_instrucao" e "cargo". Veja que enquanto grau de instrução é uma variável categórica ordinal, não podemos assumir a priori o mesmo do cargo. Há dois principais fatores que impedem essa inferência imediata:

* Não é possível comparar o grau de importância de funcionário de diferentes setores;
* É provável, e extremamente razoável, que funcionários de diferentes setores possam ter a mesma hierarquia apesar de cargos diferentes.

Temos que para a coluna de "grau_de_instrucao", as categorias listadas são, em ordem crescente:

* Ensino fundamental incompleto
* Ensino fundamental completo
* Ensino médio incompleto
* Ensino médio completo
* Ensino superior incompleto
* Ensino superior completo
* Pós graduação
* Mestrado
* Doutorado

Quando concluir esta etapas, os dados atender os seguintes critérios:

* **Padronize os valores dentro das colunas**: 
  + Valores do tipo texto devem ter a primeira letra de uma palavra no modo maiúsculo;
  + Remova espaços antes e depois de todas as colunas do tipo string;
  + Colunas de data devem estar no formato date;
  + Salário deve estar no formato double ou float.

In [0]:
# Investigando categoria de grau de instrução
display(df.groupBy('grau_de_instrucao').count().orderBy('count'))

grau_de_instrucao,count
doutorado,7
ensino fundamental completo,10
graduação,11
ensino superior completo,11
ensino medio incompleto,11
mestrado,11
ensino médio comp,12
pós graduação,12
ensino médio completo,12
4ª série completo,12


Em um cenário ideal, todos nossos dados teriam as seguintes principais características:

* Precisão - os dados são livres de erro;
* Acessibilidade - os dados são facilmente acessados;
* Abrangência  - todos os dado utilizados estão presentes;
* Consistência - os dados são confiáveis;
* Atuais - os dados são atuais;
* Relevância - quão relevantes são os dados dado seu uso.

Há algumas outras características possíveis de serem listadas, porém quero me ater ao aspecto de precisão. Ter um dado preciso é algo consideravelmente difícil, consequentemente todo dado coletado é importate. Desta forma, é possível corrigir a imprecisão entre os valores de "ensino medio incompleto" e "ensino médio comp" corrigindo as observações que possuem a grafia errada. Desta forma, estaremos preservando 12 observações de serem eliminadas quando há uma clara correção apropriada a ser feita. Analogamente, temos a presença do valor "4ª série completo", que categoriza-se enquanto "Ensino fundamental incompleto".

Antes de fazer essa correção, irei preparar string para evitar que espaços em brancos em excesso ou caixas altas/baixas interfiram na condição. Assim, temos:

In [0]:
# Função temporária para por a primeira letra maiúscula
capitalize = udf(lambda x: str(x).capitalize(), StringType())

In [0]:
# Corrigindo imprecisão do ensino médio na coluna de grau de instrução
df = df.withColumn("grau_de_instrucao", capitalize(trim(col("grau_de_instrucao") )))

# Corringdo grafias 
df = df.withColumn("grau_de_instrucao", \
              when( col("grau_de_instrucao") == 'Ensino médio comp' , "Ensino médio completo") \
                  .otherwise( when( col("grau_de_instrucao") == "Ensino medio incompleto" , "Ensino médio incompleto") \
                  .otherwise( when( col("grau_de_instrucao") == "4ª série completo", "Ensino fundamental incompleto") \
                  .otherwise(df['grau_de_instrucao']))))

# Reinvestigando categorias
display(df.groupBy('grau_de_instrucao').count().orderBy('count'))

grau_de_instrucao,count
Doutorado,7
Ensino fundamental completo,10
Ensino superior completo,11
Ensino médio incompleto,11
Graduação,11
Mestrado,11
Ensino fundamental incompleto,12
Pós graduação,12
Ensino médio completo,24


In [0]:
# Investigando categoria de cargo
display(df.groupBy('cargo').count().orderBy('count'))

cargo,count
analista de sistemas pleno,4
cientista de dados,7
analista de sistemas senior,7
líder de tecnologia,11
gerente de produção,11
assistente administrativo,12
analista de BI,12
assistente de rh,12
operador de colheita II,14
operador de colheita I,19


Não há aparente inconsistência entre os valores de cargo, com isso, irei apenas fazer o ajuste de espaços em brancos e letra inicial maiúscula.

In [0]:
# Corrigindo imprecisão do ensino médio na coluna de cargos
df = df.withColumn("cargo", capitalize(trim(col("cargo") )))

# Reverificando os cargos
display(df.groupBy('cargo').count().orderBy('count'))

cargo,count
Analista de sistemas pleno,4
Cientista de dados,7
Analista de sistemas senior,7
Líder de tecnologia,11
Gerente de produção,11
Assistente administrativo,12
Analista de bi,12
Assistente de rh,12
Operador de colheita ii,14
Operador de colheita i,19


Finalmente, todos os dados nativos estão prontos para serem transformados.

# Transformação dos dados

----

* **Admitido no mês**: essa coluna deverá ser do tipo numérico, respeitando a seguinte condição: se a data de admissão do registro for igual ao mês de referência analisado, admitido no mês recebe valor é igual a 1, senão recebe valor igual a 0.

In [0]:
# Criando coluna admitido_no_mes
df = df.withColumn("admitido_no_mes", when( (month(col('mes_referencia')) == month(col('data_de_admissao'))) \
                                             &  \
                                            (year(col('mes_referencia')) == year(col('data_de_admissao'))), 1).otherwise(0))

# Sumarizando as informações
df.groupBy("admitido_no_mes") \
    .count() \
    .withColumn("perc", col("count")*100/109).show()

+---------------+-----+------------------+
|admitido_no_mes|count|              perc|
+---------------+-----+------------------+
|              1|    1|0.9174311926605505|
|              0|  108| 99.08256880733946|
+---------------+-----+------------------+



Apenas 1 registro foi contratado no mês de referência.

-----

* **Demitido no mês**: essa coluna deverá ser do tipo numérico, respeitando a seguinte condição: se a data de demissão do registro for igual ao mês de referência analisado, demitido no mês recebe valor é igual a 1, senão recebe valor igual a 0.

In [0]:
# Criando coluna demitido_no_mes
df = df.withColumn("demitido_no_mes", when( (month(col('mes_referencia')) == month(col('data_de_demissao'))) \
                                             &  \
                                            (year(col('mes_referencia')) == year(col('data_de_demissao'))), 1).otherwise(0))

# Sumarizando as informações
df.groupBy("demitido_no_mes") \
    .count() \
    .withColumn("perc", col("count")*100/109).show()

+---------------+-----+------------------+
|demitido_no_mes|count|              perc|
+---------------+-----+------------------+
|              1|    1|0.9174311926605505|
|              0|  108| 99.08256880733946|
+---------------+-----+------------------+



Similarmente ao admitido no mês, apenas 1 registro possui uma demissão no mesmo mês de referência. Veja, **esta informação é redundate**, dado que se qualquer indivíduo for demitido da empresa, o seu último registro será precisamente o do mês de referência da demissão. Em outras palavras, sob condições normais, é impossível um indivíduo ser demitido e continuar trabalhando.

------

* **Idade**: calcule a idade em anos no mês de referência em relação à data de nascimento.

In [0]:
# Calculando idade dos indivíduos
df = df.withColumn('idade', floor(months_between(col('mes_referencia'), col('data_de_nascimento'))/12))

display(df)

mes_referencia,id_funcionario,data_de_nascimento,data_de_admissao,data_de_demissao,grau_de_instrucao,cargo,salario,admitido_no_mes,demitido_no_mes,idade
2020-01-01,11223,1995-04-01,2018-06-01,null,Ensino fundamental incompleto,Operador de colheita i,1500.0,0,0,24
2020-01-01,13254,1977-05-22,2005-09-03,null,Ensino médio incompleto,Operador de colheita i,1625.0,0,0,42
2020-01-01,13456,1960-03-05,2002-02-11,null,Ensino médio completo,Assistente de rh,1400.0,0,0,59
2020-01-01,24457,2000-07-26,2020-05-10,null,Mestrado,Gerente de produção,11500.0,0,0,19
2020-01-01,46457,1982-02-12,1998-11-07,null,Pós graduação,Analista de bi,5900.0,0,0,37
2020-01-01,46578,1972-01-11,2009-11-26,null,Ensino superior completo,Analista de sistemas pleno,7500.0,0,0,47
2020-01-01,57687,1991-05-28,2015-10-18,null,Doutorado,Cientista de dados,7200.0,0,0,28
2020-01-01,59393,1998-05-17,2019-01-08,null,Ensino médio completo,Assistente administrativo,1400.0,0,0,21
2020-01-01,95847,1993-04-23,2012-04-25,null,Graduação,Líder de tecnologia,11800.0,0,0,26
2020-02-01,11223,1995-04-01,2018-06-01,null,Ensino fundamental incompleto,Operador de colheita i,1500.0,0,0,24


------

* **Escolaridade categorizada**: com base na coluna grau de instrução, categorize a escolaridade do colaborador para conter apenas as seguintes categorias: ensino fundamental incompleto, ensino fundamental completo, ensino médio incompleto, ensino médio completo, ensino superior incompleto, ensino superior completo, pós graduação, mestrado, doutorado.

Esta etapa foi contemplada na etapa de **Padronização de Categorias**, os valores já estão devidamente ajustados. Esta etapa foi antecipada dado que "colunas de categorias, não podem conter outros valores além das categorias previstas para ela". Assim, uma vez identificado as categorias desejadas, o ajuste foi antecipado para  o 2º passo do desafio.

In [0]:
# Reinvestigando categorias
display(df.groupBy('grau_de_instrucao').count().orderBy('count'))

grau_de_instrucao,count
Doutorado,7
Ensino fundamental completo,10
Ensino superior completo,11
Ensino médio incompleto,11
Graduação,11
Mestrado,11
Ensino fundamental incompleto,12
Pós graduação,12
Ensino médio completo,24


* **Tempo de empresa**: com base no mês referência e a data de admissão, obtenha o tempo de empresa do colaborador em meses.

In [0]:
# Calculando tempo de empresa do colaborador em meses
df = df.withColumn('tempo_de_empresa', round(months_between(col('mes_referencia'), col('data_de_admissao'))))

display(df)


mes_referencia,id_funcionario,data_de_nascimento,data_de_admissao,data_de_demissao,grau_de_instrucao,cargo,salario,admitido_no_mes,demitido_no_mes,idade,tempo_de_empresa
2020-01-01,11223,1995-04-01,2018-06-01,null,Ensino fundamental incompleto,Operador de colheita i,1500.0,0,0,24,19.0
2020-01-01,13254,1977-05-22,2005-09-03,null,Ensino médio incompleto,Operador de colheita i,1625.0,0,0,42,172.0
2020-01-01,13456,1960-03-05,2002-02-11,null,Ensino médio completo,Assistente de rh,1400.0,0,0,59,215.0
2020-01-01,24457,2000-07-26,2020-05-10,null,Mestrado,Gerente de produção,11500.0,0,0,19,-4.0
2020-01-01,46457,1982-02-12,1998-11-07,null,Pós graduação,Analista de bi,5900.0,0,0,37,254.0
2020-01-01,46578,1972-01-11,2009-11-26,null,Ensino superior completo,Analista de sistemas pleno,7500.0,0,0,47,121.0
2020-01-01,57687,1991-05-28,2015-10-18,null,Doutorado,Cientista de dados,7200.0,0,0,28,50.0
2020-01-01,59393,1998-05-17,2019-01-08,null,Ensino médio completo,Assistente administrativo,1400.0,0,0,21,12.0
2020-01-01,95847,1993-04-23,2012-04-25,null,Graduação,Líder de tecnologia,11800.0,0,0,26,92.0
2020-02-01,11223,1995-04-01,2018-06-01,null,Ensino fundamental incompleto,Operador de colheita i,1500.0,0,0,24,20.0


In [0]:
# Estatística descritiva do tempo de empresa
df.select('tempo_de_empresa').describe().show()

+-------+------------------+
|summary|  tempo_de_empresa|
+-------+------------------+
|  count|               109|
|   mean|112.65137614678899|
| stddev| 86.79022988054768|
|    min|              -4.0|
|    max|             265.0|
+-------+------------------+



Perceba que há um tempo de empresa negativo (-4), isto não deveria ocorrer e é um sinal de que alguma informação inconsistente sobreviveu a etapa de limpeza. Esta imprecisão ocorre devido ao fator de existir pelo menos 1 observação ao qual a data de admissão é posterior ao mês de referência. Existem duas possíveis soluções:

* Corrigir o mês de referência do indivíduo
* Retirar a observação

Adotarei a remoção do indivíduo pois não há garantias que de aquela observação possui mais tempo de casa do que zero - o que aconteceria caso o mês de referência fosse alterado.

In [0]:
# Verificando observações com tempo de empresa negativo
df.filter(col('tempo_de_empresa') < 0).show()

+--------------+--------------+------------------+----------------+----------------+-----------------+-------------------+-------+---------------+---------------+-----+----------------+
|mes_referencia|id_funcionario|data_de_nascimento|data_de_admissao|data_de_demissao|grau_de_instrucao|              cargo|salario|admitido_no_mes|demitido_no_mes|idade|tempo_de_empresa|
+--------------+--------------+------------------+----------------+----------------+-----------------+-------------------+-------+---------------+---------------+-----+----------------+
|    2020-01-01|         24457|        2000-07-26|      2020-05-10|            null|         Mestrado|Gerente de produção|11500.0|              0|              0|   19|            -4.0|
|    2020-03-01|         24457|        2000-07-26|      2020-05-10|            null|         Mestrado|Gerente de produção|11500.0|              0|              0|   19|            -2.0|
|    2020-04-01|         24457|        2000-07-26|      2020-05-10|   

In [0]:
df = df.filter(col('tempo_de_empresa') > 0)

Note que é apenas um funcionário que está causando problemas. **Recomenda-se a investigação da data de admissão do funcionário 24457**, por hora iremos removê-lo dos dados.

-----

* **Mudou de salário:** compare o salário do mês atual com o salário do mês anterior dos funcionários, se houve mudança atribua 1 como valor para a coluna do mês atual, senão atribua 0.

In [0]:
# Criando coluna de identificação de mudança de salário
df = df.withColumn('salario_lag', lag(col('salario')).over(Window.partitionBy("id_funcionario").orderBy('mes_referencia'))) \
       .withColumn('mudou_salario', when(col('salario') != col('salario_lag'), 1).otherwise(0))


In [0]:
# Removendo coluna auxiliar de salário lag
df = df.drop('salario_lag')

display(df)

mes_referencia,id_funcionario,data_de_nascimento,data_de_admissao,data_de_demissao,grau_de_instrucao,cargo,salario,admitido_no_mes,demitido_no_mes,idade,tempo_de_empresa,mudou_salario
2020-01-01,11223,1995-04-01,2018-06-01,null,Ensino fundamental incompleto,Operador de colheita i,1500.0,0,0,24,19.0,0
2020-02-01,11223,1995-04-01,2018-06-01,null,Ensino fundamental incompleto,Operador de colheita i,1500.0,0,0,24,20.0,0
2020-03-01,11223,1995-04-01,2018-06-01,null,Ensino fundamental incompleto,Operador de colheita i,1500.0,0,0,24,21.0,0
2020-04-01,11223,1995-04-01,2018-06-01,null,Ensino fundamental incompleto,Operador de colheita i,1500.0,0,0,25,22.0,0
2020-05-01,11223,1995-04-01,2018-06-01,null,Ensino fundamental incompleto,Operador de colheita i,1500.0,0,0,25,23.0,0
2020-06-01,11223,1995-04-01,2018-06-01,null,Ensino fundamental incompleto,Operador de colheita i,1500.0,0,0,25,24.0,0
2020-07-01,11223,1995-04-01,2018-06-01,null,Ensino fundamental incompleto,Operador de colheita i,1500.0,0,0,25,25.0,0
2020-08-01,11223,1995-04-01,2018-06-01,null,Ensino fundamental incompleto,Operador de colheita i,1500.0,0,0,25,26.0,0
2020-09-01,11223,1995-04-01,2018-06-01,null,Ensino fundamental incompleto,Operador de colheita ii,1750.0,0,0,25,27.0,1
2020-10-01,11223,1995-04-01,2018-06-01,null,Ensino fundamental incompleto,Operador de colheita ii,1750.0,0,0,25,28.0,0


In [0]:
# Reordenando por mês de referência
df = df.orderBy('mes_referencia')

display(df)

mes_referencia,id_funcionario,data_de_nascimento,data_de_admissao,data_de_demissao,grau_de_instrucao,cargo,salario,admitido_no_mes,demitido_no_mes,idade,tempo_de_empresa,mudou_salario
2020-01-01,59393,1998-05-17,2019-01-08,null,Ensino médio completo,Assistente administrativo,1400.0,0,0,21,12.0,0
2020-01-01,95847,1993-04-23,2012-04-25,null,Graduação,Líder de tecnologia,11800.0,0,0,26,92.0,0
2020-01-01,13254,1977-05-22,2005-09-03,null,Ensino médio incompleto,Operador de colheita i,1625.0,0,0,42,172.0,0
2020-01-01,46457,1982-02-12,1998-11-07,null,Pós graduação,Analista de bi,5900.0,0,0,37,254.0,0
2020-01-01,46578,1972-01-11,2009-11-26,null,Ensino superior completo,Analista de sistemas pleno,7500.0,0,0,47,121.0,0
2020-01-01,11223,1995-04-01,2018-06-01,null,Ensino fundamental incompleto,Operador de colheita i,1500.0,0,0,24,19.0,0
2020-01-01,57687,1991-05-28,2015-10-18,null,Doutorado,Cientista de dados,7200.0,0,0,28,50.0,0
2020-01-01,13456,1960-03-05,2002-02-11,null,Ensino médio completo,Assistente de rh,1400.0,0,0,59,215.0,0
2020-02-01,57687,1991-05-28,2015-10-18,null,Doutorado,Cientista de dados,7200.0,0,0,28,51.0,0
2020-02-01,59393,1998-05-17,2019-01-08,null,Ensino médio completo,Assistente administrativo,1400.0,0,0,21,13.0,0


In [0]:
# Verificando quantidade de mudanças salariais
df.filter(col('mudou_salario') == 1).count()

Out[742]: 9

Houve 10 mudanças salariais no período analisado. Podemos verificar a quantidade de promoções por indivíduo:

In [0]:
display(df.groupBy('id_funcionario').agg(col('id_funcionario'), sum('mudou_salario').alias('total_mudancas')).orderBy('total_mudancas'))

id_funcionario,id_funcionario,total_mudancas
24457,24457,0
57687,57687,0
11223,11223,1
46457,46457,1
59393,59393,1
13254,13254,1
46578,46578,1
95847,95847,1
13456,13456,1
12345,12345,2


O funcionário 57687 não teve mudança salarial no período observado. Por outro lado, o funcionário 12345 obteve 2 mudanças salariais.

------

* **Mudou de cargo:** compare o cargo do mês atual com o cargo do mês anterior dos funcionários, se houve mudança atribua 1 como valor para a coluna do mês atual, senão atribua 0.

In [0]:
# Criando coluna de identificação de mudança de salário
df = df.withColumn('cargo_lag', lag(col('cargo')).over(Window.partitionBy("id_funcionario").orderBy('mes_referencia'))) \
       .withColumn('mudou_cargo', when(col('cargo') != col('cargo_lag'), 1).otherwise(0))

# Removendo coluna auxiliar de salário lag
df = df.drop('cargo_lag')

display(df)

mes_referencia,id_funcionario,data_de_nascimento,data_de_admissao,data_de_demissao,grau_de_instrucao,cargo,salario,admitido_no_mes,demitido_no_mes,idade,tempo_de_empresa,mudou_salario,mudou_cargo
2020-01-01,11223,1995-04-01,2018-06-01,null,Ensino fundamental incompleto,Operador de colheita i,1500.0,0,0,24,19.0,0,0
2020-02-01,11223,1995-04-01,2018-06-01,null,Ensino fundamental incompleto,Operador de colheita i,1500.0,0,0,24,20.0,0,0
2020-03-01,11223,1995-04-01,2018-06-01,null,Ensino fundamental incompleto,Operador de colheita i,1500.0,0,0,24,21.0,0,0
2020-04-01,11223,1995-04-01,2018-06-01,null,Ensino fundamental incompleto,Operador de colheita i,1500.0,0,0,25,22.0,0,0
2020-05-01,11223,1995-04-01,2018-06-01,null,Ensino fundamental incompleto,Operador de colheita i,1500.0,0,0,25,23.0,0,0
2020-06-01,11223,1995-04-01,2018-06-01,null,Ensino fundamental incompleto,Operador de colheita i,1500.0,0,0,25,24.0,0,0
2020-07-01,11223,1995-04-01,2018-06-01,null,Ensino fundamental incompleto,Operador de colheita i,1500.0,0,0,25,25.0,0,0
2020-08-01,11223,1995-04-01,2018-06-01,null,Ensino fundamental incompleto,Operador de colheita i,1500.0,0,0,25,26.0,0,0
2020-09-01,11223,1995-04-01,2018-06-01,null,Ensino fundamental incompleto,Operador de colheita ii,1750.0,0,0,25,27.0,1,1
2020-10-01,11223,1995-04-01,2018-06-01,null,Ensino fundamental incompleto,Operador de colheita ii,1750.0,0,0,25,28.0,0,0


In [0]:
# Reordenando os dados por mês de referência
df = df.orderBy('mes_referencia')

display(df)

mes_referencia,id_funcionario,data_de_nascimento,data_de_admissao,data_de_demissao,grau_de_instrucao,cargo,salario,admitido_no_mes,demitido_no_mes,idade,tempo_de_empresa,mudou_salario,mudou_cargo
2020-01-01,59393,1998-05-17,2019-01-08,null,Ensino médio completo,Assistente administrativo,1400.0,0,0,21,12.0,0,0
2020-01-01,95847,1993-04-23,2012-04-25,null,Graduação,Líder de tecnologia,11800.0,0,0,26,92.0,0,0
2020-01-01,13254,1977-05-22,2005-09-03,null,Ensino médio incompleto,Operador de colheita i,1625.0,0,0,42,172.0,0,0
2020-01-01,46457,1982-02-12,1998-11-07,null,Pós graduação,Analista de bi,5900.0,0,0,37,254.0,0,0
2020-01-01,46578,1972-01-11,2009-11-26,null,Ensino superior completo,Analista de sistemas pleno,7500.0,0,0,47,121.0,0,0
2020-01-01,11223,1995-04-01,2018-06-01,null,Ensino fundamental incompleto,Operador de colheita i,1500.0,0,0,24,19.0,0,0
2020-01-01,57687,1991-05-28,2015-10-18,null,Doutorado,Cientista de dados,7200.0,0,0,28,50.0,0,0
2020-01-01,13456,1960-03-05,2002-02-11,null,Ensino médio completo,Assistente de rh,1400.0,0,0,59,215.0,0,0
2020-02-01,57687,1991-05-28,2015-10-18,null,Doutorado,Cientista de dados,7200.0,0,0,28,51.0,0,0
2020-02-01,59393,1998-05-17,2019-01-08,null,Ensino médio completo,Assistente administrativo,1400.0,0,0,21,13.0,0,0


In [0]:
# Verificando quantidade de mudanças de cargo
df.filter(col('mudou_cargo') == 1).count()

Out[746]: 2

In [0]:
display(df.groupBy('id_funcionario').agg(col('id_funcionario'), sum('mudou_cargo').alias('total_mudancas')).orderBy('total_mudancas'))

id_funcionario,id_funcionario,total_mudancas
13456,13456,0
13254,13254,0
12345,12345,0
57687,57687,0
59393,59393,0
95847,95847,0
24457,24457,0
46457,46457,0
11223,11223,1
46578,46578,1


Os funcionário 11223 e 46578 tiveram mudanças de cargo no período analisado.

## Resumo dos dados após limpeza e transformações

In [0]:
# Descrição final após limpeza dos dados
print(f"Nº observações final: {df.count()}")
obs_perdidas = df_original.count() - df.count()  
perc_perda = (1 - df.count()/df_original.count())*100
print(f"Observações perdidas: {obs_perdidas}\nPercentual de perda de observações: {perc_perda:.3}%")
print("\n")

display(df)

print("\n")

display(df.describe())

print("\n")

df.printSchema()

Nº observações final: 105
Observações perdidas: 16
Percentual de perda de observações: 13.2%




mes_referencia,id_funcionario,data_de_nascimento,data_de_admissao,data_de_demissao,grau_de_instrucao,cargo,salario,admitido_no_mes,demitido_no_mes,idade,tempo_de_empresa,mudou_salario,mudou_cargo
2020-01-01,59393,1998-05-17,2019-01-08,null,Ensino médio completo,Assistente administrativo,1400.0,0,0,21,12.0,0,0
2020-01-01,95847,1993-04-23,2012-04-25,null,Graduação,Líder de tecnologia,11800.0,0,0,26,92.0,0,0
2020-01-01,13254,1977-05-22,2005-09-03,null,Ensino médio incompleto,Operador de colheita i,1625.0,0,0,42,172.0,0,0
2020-01-01,46457,1982-02-12,1998-11-07,null,Pós graduação,Analista de bi,5900.0,0,0,37,254.0,0,0
2020-01-01,46578,1972-01-11,2009-11-26,null,Ensino superior completo,Analista de sistemas pleno,7500.0,0,0,47,121.0,0,0
2020-01-01,11223,1995-04-01,2018-06-01,null,Ensino fundamental incompleto,Operador de colheita i,1500.0,0,0,24,19.0,0,0
2020-01-01,57687,1991-05-28,2015-10-18,null,Doutorado,Cientista de dados,7200.0,0,0,28,50.0,0,0
2020-01-01,13456,1960-03-05,2002-02-11,null,Ensino médio completo,Assistente de rh,1400.0,0,0,59,215.0,0,0
2020-02-01,57687,1991-05-28,2015-10-18,null,Doutorado,Cientista de dados,7200.0,0,0,28,51.0,0,0
2020-02-01,59393,1998-05-17,2019-01-08,null,Ensino médio completo,Assistente administrativo,1400.0,0,0,21,13.0,0,0


summary,id_funcionario,grau_de_instrucao,cargo,salario,admitido_no_mes,demitido_no_mes,idade,tempo_de_empresa,mudou_salario,mudou_cargo
count,105,105,105,105,105,105,105,105,105,105
mean,37878.80952380953,null,null,4977.571428571428,0.0,0.009523809523809525,35.63809523809524,117.00952380952381,0.08571428571428572,0.01904761904761905
stddev,27230.28388404493,null,null,4076.539003524037,0.0,0.09759000729485331,12.480079364827008,85.43784236756514,0.28128433856309726,0.13734798456646122
min,11223,Doutorado,Analista de bi,1400.0,0,0,19,1.0,0,0
max,95847,Pós graduação,Operador de colheita ii,12500.0,0,1,60,265.0,1,1




root
 |-- mes_referencia: date (nullable = true)
 |-- id_funcionario: string (nullable = true)
 |-- data_de_nascimento: date (nullable = true)
 |-- data_de_admissao: date (nullable = true)
 |-- data_de_demissao: date (nullable = true)
 |-- grau_de_instrucao: string (nullable = true)
 |-- cargo: string (nullable = true)
 |-- salario: double (nullable = true)
 |-- admitido_no_mes: integer (nullable = false)
 |-- demitido_no_mes: integer (nullable = false)
 |-- idade: long (nullable = true)
 |-- tempo_de_empresa: double (nullable = true)
 |-- mudou_salario: integer (nullable = false)
 |-- mudou_cargo: integer (nullable = false)

